In [ ]:
import pandas as pd 
import json
import pysam

In [ ]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230220beida/cr7/WLR-T-T-T/03.assemble/WLR-T-T-T/outs/all_contig_annotations.csv")

In [ ]:
df_filter = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230220beida/cr7/WLR-T-T-T/03.assemble/WLR-T-T-T/outs/filtered_contig_annotations.csv")

In [ ]:
df_productive = df[df["productive"]==True]

In [ ]:
df_productive

In [ ]:
df_cell = df_productive[df_productive["is_cell"]==True]

In [ ]:
df_cell

In [ ]:
len(set(df_cell.barcode))

In [ ]:
df_call = df_productive[df_productive["is_cell"]==False]

In [ ]:
df_call

In [ ]:
# 方案一 calling is_cell=False的细胞 条件 
# 1.满足productive=True 
# 2.high_confidence=True 
# 3.groupby barcode chain，取最高umi的链 
# 4.作为calling回的细胞合并至filtered annotation文件
df_call_1 = df_call[df_call["high_confidence"]==True]

In [ ]:
df_call_1 = df_call_1.sort_values("umis", ascending=False)

In [ ]:
df_call_1 = df_call_1.groupby(["barcode", "chain"], as_index=False).head(1)

In [ ]:
len(set(df_call_1.barcode))

In [ ]:
df_call_1

In [ ]:
df_call_1.sort_values("barcode")

In [ ]:
df_call_1 = df_call_1.groupby("barcode").filter(lambda x: (len(x) > 1))

In [ ]:
df_filter

In [ ]:
df_merge = pd.concat([df_filter,df_call_1])

In [ ]:
with open("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230220beida/cr7/WLR-T-T-T/02.convert/barcode_convert.json", 'r') as f:
    tenX_sgr = json.load(f)

In [ ]:
df_merge['barcode'] = df_merge['barcode'].apply(lambda x: tenX_sgr[x.split('-')[0]])
df_merge['contig_id'] = df_merge['contig_id'].apply(lambda x: tenX_sgr[x.split('-')[0]]+'_'+x.split('_')[1]+'_'+x.split('_')[2])

In [ ]:
df_merge.to_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230321beida_cellcall/test/WLR-T-T-T/04.summarize/filtered_contig_annotations.csv", sep=',', index=False)

In [ ]:
len(set(df_merge.barcode))

In [ ]:
count = 0
total_reads = 0
target = set(df_filter.barcode)
with pysam.AlignmentFile("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230321beida_cellcall/WLR-T-T-T/03.assemble/WLR-T-T-T/outs/all_contig.bam") as f:
    for read in f:
        total_reads+=1
        cb = read.get_tag('CB')
        if cb in target:
            count += 1

In [ ]:
count

In [ ]:
total_reads

In [ ]:
# 方案二 is_cell=False的细胞 calling条件 1.满足productive=True 2.high_confidence=True 3.同时拥有ab链，取最高umi的链
import pandas as pd 
import os
import glob
import pysam
import numpy as np
import json
from celescope.tools import utils

In [ ]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230220beida/cr7/WLR-T-T-T/03.assemble/WLR-T-T-T/outs/all_contig_annotations.csv")

In [ ]:
with open("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230321beida_cellcall/WLR-T-T-T/02.convert/barcode_convert.json", 'r') as f:
    tenX_sgr = json.load(f)

In [ ]:
match_cell_barcodes, _ = utils.get_barcode_from_match_dir("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230220beida/match_dir/WLR-T/")

In [ ]:
df['barcode'] = df['barcode'].apply(lambda x: tenX_sgr[x.split('-')[0]])

In [ ]:
df_match = df[df["barcode"].isin(match_cell_barcodes)]

In [ ]:
df_match = df_match[df_match["is_cell"]==True]

In [ ]:
len(set(df_match.barcode))

In [ ]:
len(set(df_cell.barcode))

In [ ]:
df = df[df["productive"]==True]
df_cell = df[df["is_cell"]==True]
df_not_cell = df[df["is_cell"]==False]

In [ ]:
df_not_cell = df_not_cell.sort_values('umis',ascending=False)

In [ ]:
df_not_cell

In [ ]:
group_elements = ['barcode', 'chain']

In [ ]:
df_not_cell = df_not_cell.drop_duplicates(group_elements)

In [ ]:
len(set(df_not_cell.barcode))

In [ ]:
df_not_cell = df_not_cell.groupby("barcode").filter(lambda x: (len(x) > 1))

In [ ]:
len(set(df_not_cell.barcode))

In [ ]:
df_not_cell.sort_values(['barcode','umis'],ascending=[False,False])

In [ ]:
df_not_cell_match = df_not_cell[df_not_cell["barcode"].isin(match_cell_barcodes)]

In [ ]:
len(set(df_not_cell_match.barcode))

In [ ]:
# SNR
df_not_cell

In [ ]:
bc_chain_dict = df_not_cell.groupby("barcode")["chain"].apply(lambda x: set(x)).to_dict()

In [ ]:
pair_chain_dict = {key: value for key, value in bc_chain_dict.items() if len(value)==2}

In [ ]:
pair_chain_dict # 同时含有TRA TRB的barcode

In [ ]:
df_not_cell = df_not_cell[df_not_cell["barcode"].isin(pair_chain_dict)]

In [ ]:
df_not_cell = df_not_cell.sort_values(["barcode","umis"], ascending=[False, False])

In [ ]:
df_not_cell

In [ ]:
df_not_cell_multi_chain = df_not_cell.groupby("barcode").filter(lambda x: (len(x) > 2)) # 有多条TRA或TRB的barcode
df_not_cell_pair_chain = df_not_cell.groupby("barcode").filter(lambda x: (len(x) == 2))# 有两条TRA或TRB的barcode

In [ ]:
len(set(df_not_cell_pair_chain.barcode))

In [ ]:
df_not_cell_multi_chain

In [ ]:
# 对多条TRA或TRB的barcode SNR
df_tra = df_not_cell_multi_chain[df_not_cell_multi_chain['chain'] == 'TRA']
df_trb = df_not_cell_multi_chain[df_not_cell_multi_chain['chain'] == 'TRB']

In [ ]:
tra_dict = df_tra.groupby("barcode")["umis"].apply(lambda x: x.tolist()).to_dict()
trb_dict = df_trb.groupby("barcode")["umis"].apply(lambda x: x.tolist()).to_dict()

In [ ]:
len(tra_dict)

In [ ]:
len(trb_dict)

In [ ]:
len(tra_dict.keys() & trb_dict.keys())

In [ ]:
tra_dict['AAACATCGAAACATCGCCGACAAC']

In [ ]:
for k in tra_dict:
    if len(tra_dict[k]) == 1:
        tra_dict[k].append(0.1)
for k in trb_dict:
    if len(trb_dict[k]) == 1:
        trb_dict[k].append(0.1)

In [ ]:
tra_dict

In [ ]:
filter_noise_tra_dict = {key for key,value in tra_dict.items() if value[0]/value[1] >= 2}
filter_noise_trb_dict = {key:value for key,value in trb_dict.items() if value[0]/value[1] >= 2}

In [ ]:
len(filter_noise_trb_dict)

In [ ]:
len(filter_noise_tra_dict.keys() & filter_noise_trb_dict.keys())

In [ ]:
for k in tra_dict:
    if len(tra_dict[k]) == 1:
        tra_dict[k].append(tra_dict[k][0])
for k in trb_dict:
    if len(trb_dict[k]) == 1:
        trb_dict[k].append(trb_dict[k][0])

In [ ]:
trb_dict

In [ ]:
filter_noise_tra = set()
filter_noise_trb = set()
for k, v in tra_dict.items():
    S = v[0]
    noise = v[1:]
    B = np.mean(noise)
    O = np.std(noise)
    if O == 0:
        filter_noise_tra.add(k)
        continue
    SNR = (S-B)/O
    if SNR >= 10:
        filter_noise_tra.add(k)
for k, v in trb_dict.items():
    S = v[0]
    noise = v[1:]
    B = np.mean(noise)
    O = np.std(noise)
    if O == 0:
        filter_noise_trb.add(k)
        continue
    SNR = (S-B)/O
    if SNR >= 10:
        filter_noise_trb.add(k)

In [ ]:
len(filter_noise_tra & filter_noise_trb)

In [ ]:
np.std([4, 4])

In [ ]:
# (S − B)/σ
S = 10
B = np.mean([4,4,2,1])
O = np.median([4,4,2,1])

In [ ]:
SNR = (S-B)/O

In [ ]:
SNR

In [22]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20230309ningxiaBCR_downsample/5g/SS_02/03.assemble/SS_02/outs/all_contig_annotations.csv")
df = df[df["productive"]==True]
df_cell = df[df["is_cell"]==True]
df_not_cell = df[df["is_cell"]==False]
df_not_cell

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,...,fwr3_nt,cdr3,cdr3_nt,fwr4,fwr4_nt,reads,umis,raw_clonotype_id,raw_consensus_id,exact_subclonotype_id
11,AAACCTGAGAAACGAG-1,False,AAACCTGAGAAACGAG-1_contig_1,False,523,IGL,IGLV3-1,NaN,IGLJ2,IGLC2,...,NaN,CQAWDSSTYVVF,TGTCAGGCGTGGGACAGCAGCACCTATGTGGTATTC,NaN,NaN,1131,25,NaN,NaN,NaN
12,AAACCTGAGAAACGAG-1,False,AAACCTGAGAAACGAG-1_contig_2,False,445,IGL,IGLV3-19,NaN,IGLJ1,IGLC2,...,NaN,CNSRDSSGNPLEVF,TGTAACTCCCGGGACAGCAGTGGTAACCCTCTAGAGGTCTTC,NaN,NaN,551,13,NaN,NaN,NaN
13,AAACCTGAGAAACGAG-1,False,AAACCTGAGAAACGAG-1_contig_3,False,440,IGH,IGHV7-4-1,NaN,IGHJ4,NaN,...,NaN,CARGAAADYW,TGTGCGAGAGGGGCAGCAGCAGATTACTGG,NaN,NaN,123,32,NaN,NaN,NaN
14,AAACCTGAGAAACGAG-1,False,AAACCTGAGAAACGAG-1_contig_4,False,451,IGL,IGLV7-46,NaN,IGLJ3,IGLC2,...,NaN,CLVSYSGAWVF,TGCTTGGTCTCCTATAGTGGTGCTTGGGTGTTC,NaN,NaN,34,7,NaN,NaN,NaN
15,AAACCTGAGAAACGAG-1,False,AAACCTGAGAAACGAG-1_contig_5,False,530,IGL,IGLV3-21,NaN,IGLJ2,IGLC2,...,NaN,CQVWDGNSDYFVF,TGTCAGGTGTGGGATGGTAACAGTGATTATTTTGTATTC,NaN,NaN,534,133,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176592,GAGTCCGGTAGAAAGG-1,False,GAGTCCGGTAGAAAGG-1_contig_1,False,436,IGK,IGKV3-20,NaN,IGKJ4,IGKC,...,NaN,CQQYGRSPLTF,TGTCAGCAGTATGGTAGGTCACCCCTCACTTTC,NaN,NaN,7,1,NaN,NaN,NaN
176605,GATCGTACACTCAGGC-1,False,GATCGTACACTCAGGC-1_contig_1,False,491,IGK,IGKV3-11,NaN,IGKJ4,IGKC,...,NaN,CQQRSDWPPLTF,TGTCAGCAGCGTAGCGACTGGCCTCCACTCACTTTC,NaN,NaN,11,1,NaN,NaN,NaN
176608,GATGAAACAGCTCGAC-1,False,GATGAAACAGCTCGAC-1_contig_1,False,452,IGK,IGKV2D-28,NaN,IGKJ2,IGKC,...,NaN,CMQALQAPYTF,TGCATGCAAGCTCTACAAGCTCCTTACACTTTT,NaN,NaN,7,1,NaN,NaN,NaN
176613,GATGAGGAGAGGTTAT-1,False,GATGAGGAGAGGTTAT-1_contig_1,False,397,IGL,IGLV3-21,NaN,IGLJ2,NaN,...,NaN,CQVWDGISDYLVF,TGTCAGGTGTGGGATGGTATCAGTGATTATTTGGTATTC,NaN,NaN,5,1,NaN,NaN,NaN


In [47]:
bc_chain_dict = df.groupby("barcode")["chain"].apply(lambda x: set(x)).to_dict()

In [ ]:
bc_chain_dict

In [48]:
pair_chain_dict = {key: value for key, value in bc_chain_dict.items() if len(value)>=2}

In [32]:
pair_chain_dict["AAACCTGTCCGTACAA-1"]

KeyError: 'AAACCTGTCCGTACAA-1'

In [ ]:
pair_chain_dict = {key: value for key, value in bc_chain_dict.items() if len(value)>=2 and "IGH" in value}
pair_chain_dict

In [50]:
len(pair_chain_dict)

10960

In [45]:
pair_chain_dict = {key: value for key, value in pair_chain_dict.items() if "IGH" in value}